In [1]:
import math
import random

# Inisialisasi Papan
board = [' ' for x in range(9)] # 9 slot: 0-8

HUMAN = 'X'
COMPUTER = 'O'
EMPTY = ' '

# --- Fungsi Tampilan dan Utilitas Papan ---

def print_board(board):
    """Menampilkan papan Tic-Tac-Toe 3x3."""
    print('-------------')
    # Mencetak 3 baris
    for i in range(3):
        print(f'| {board[i*3]} | {board[i*3 + 1]} | {board[i*3 + 2]} |')
        print('-------------')

def check_win(board, player):
    """Memeriksa apakah pemain tertentu menang."""
    # Baris, Kolom, Diagonal
    win_conditions = [
        # Baris
        [0, 1, 2], [3, 4, 5], [6, 7, 8],
        # Kolom
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        # Diagonal
        [0, 4, 8], [2, 4, 6]
    ]

    for condition in win_conditions:
        if all(board[i] == player for i in condition):
            return True
    return False

def get_available_moves(board):
    """Mengembalikan daftar indeks dari sel kosong."""
    return [i for i, spot in enumerate(board) if spot == EMPTY]

def is_board_full(board):
    """Memeriksa apakah papan penuh (seri)."""
    return EMPTY not in board

def get_game_state(board):
    """
    Mengembalikan:
    1: HUMAN (X) menang
    -1: COMPUTER (O) menang
    0: Seri
    None: Permainan berlanjut
    """
    if check_win(board, HUMAN):
        return 1
    elif check_win(board, COMPUTER):
        return -1
    elif is_board_full(board):
        return 0
    return None

# --- Algoritma Minimax dengan Alpha-Beta Pruning ---

def minimax(board, depth, alpha, beta, maximizing_player):
    """
    Algoritma Minimax dengan optimasi Alpha-Beta Pruning.

    :param board: Kondisi papan saat ini.
    :param depth: Kedalaman pencarian (makin dalam makin baik).
    :param alpha: Nilai terbaik yang ditemukan sejauh ini untuk Maximizer (Awalnya -inf).
    :param beta: Nilai terbaik yang ditemukan sejauh ini untuk Minimizer (Awalnya +inf).
    :param maximizing_player: Boolean, True jika pemain saat ini adalah Maximizer (Manusia/X).
    :return: (skor, langkah_terbaik)
    """
    # 1. Kasus Basis: Periksa kondisi terminal (menang, kalah, atau seri)
    state = get_game_state(board)
    if state is not None:
        return state, None

    # Jika Maximizing Player (Manusia/X)
    if maximizing_player:
        max_eval = -math.inf
        best_move = None

        for move in get_available_moves(board):
            # Lakukan langkah (Simulasi)
            board[move] = HUMAN

            # Panggilan rekursif untuk Minimizer (Komputer/O)
            evaluation, _ = minimax(board, depth + 1, alpha, beta, False)

            # Batalkan langkah (Backtracking)
            board[move] = EMPTY

            if evaluation > max_eval:
                max_eval = evaluation
                best_move = move

            # Alpha-Beta Pruning: Pemangkasan
            # Jika nilai terbaik Max (alpha) lebih besar/sama dengan nilai terbaik Min (beta)
            alpha = max(alpha, max_eval)
            if beta <= alpha:
                break

        return max_eval, best_move

    # Jika Minimizing Player (Komputer/O)
    else:
        min_eval = math.inf
        best_move = None

        for move in get_available_moves(board):
            # Lakukan langkah (Simulasi)
            board[move] = COMPUTER

            # Panggilan rekursif untuk Maximizer (Manusia/X)
            evaluation, _ = minimax(board, depth + 1, alpha, beta, True)

            # Batalkan langkah (Backtracking)
            board[move] = EMPTY

            if evaluation < min_eval:
                min_eval = evaluation
                best_move = move

            # Alpha-Beta Pruning: Pemangkasan
            # Jika nilai terbaik Min (beta) lebih kecil/sama dengan nilai terbaik Max (alpha)
            beta = min(beta, min_eval)
            if beta <= alpha:
                break

        return min_eval, best_move

# --- Fungsi Utama Permainan ---

def play_game():
    """Mengatur alur permainan Tic-Tac-Toe."""
    current_board = [' ' for x in range(9)]
    current_player = HUMAN # Manusia (X) mulai duluan

    print("Selamat datang di Tic-Tac-Toe!")
    print("Anda adalah 'X' dan Komputer adalah 'O'.")
    print("Pilih langkah Anda dengan memasukkan angka 1-9 (sesuai posisi pada papan).")
    print("Peta posisi:")
    print("-------------")
    print("| 1 | 2 | 3 |")
    print("| 4 | 5 | 6 |")
    print("| 7 | 8 | 9 |")
    print("-------------")

    while get_game_state(current_board) is None:
        print_board(current_board)

        if current_player == HUMAN:
            # Langkah Manusia
            while True:
                try:
                    move_input = input("Giliran Anda (X). Masukkan langkah (1-9): ")
                    move = int(move_input) - 1 # Ubah 1-9 menjadi indeks 0-8

                    if 0 <= move <= 8 and current_board[move] == EMPTY:
                        current_board[move] = HUMAN
                        break
                    else:
                        print("Langkah tidak valid. Coba lagi.")
                except ValueError:
                    print("Input tidak valid. Masukkan angka.")

        else:
            # Langkah Komputer (Menggunakan Minimax + Alpha-Beta Pruning)
            print("Giliran Komputer (O)...")

            # Panggil Minimax. Komputer adalah Minimizer, jadi maximizing_player=False
            # Alpha = -inf, Beta = +inf
            score, best_move = minimax(current_board, 0, -math.inf, math.inf, False)

            if best_move is not None:
                current_board[best_move] = COMPUTER
            # Jika tidak ada best_move (ini jarang terjadi di TTT, tapi sebagai safety)
            elif get_available_moves(current_board):
                 current_board[get_available_moves(current_board)[0]] = COMPUTER # Ambil langkah kosong pertama

        # Tukar pemain
        current_player = COMPUTER if current_player == HUMAN else HUMAN

    # Permainan Selesai
    print_board(current_board)
    final_state = get_game_state(current_board)

    if final_state == 1:
        print("🎉 Selamat! Anda (X) menang!")
    elif final_state == -1:
        print("🤖 Komputer (O) menang! Maaf, komputer bermain optimal.")
    else:
        print("🤝 Permainan berakhir seri!")

# Mulai permainan
if __name__ == "__main__":
    play_game()

Selamat datang di Tic-Tac-Toe!
Anda adalah 'X' dan Komputer adalah 'O'.
Pilih langkah Anda dengan memasukkan angka 1-9 (sesuai posisi pada papan).
Peta posisi:
-------------
| 1 | 2 | 3 |
| 4 | 5 | 6 |
| 7 | 8 | 9 |
-------------
-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
Giliran Anda (X). Masukkan langkah (1-9): 1
-------------
| X |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
Giliran Komputer (O)...
-------------
| X |   |   |
-------------
|   | O |   |
-------------
|   |   |   |
-------------
Giliran Anda (X). Masukkan langkah (1-9): 2
-------------
| X | X |   |
-------------
|   | O |   |
-------------
|   |   |   |
-------------
Giliran Komputer (O)...
-------------
| X | X | O |
-------------
|   | O |   |
-------------
|   |   |   |
-------------
Giliran Anda (X). Masukkan langkah (1-9): 4
-------------
| X | X | O |
-------------
| X | O |   |
-------------
|   |   |   |
-------------
Gi